In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split 

In [3]:
raw_train_data = pd.read_csv('../input/digit-recognizer/train.csv')
raw_test_data = pd.read_csv('../input/digit-recognizer/test.csv')

In [4]:
train_data=raw_train_data.copy()
test_data=raw_test_data.copy()

In [5]:
IMG_SIZE=28

train_data_input = train_data.iloc[:,1:]
train_data_input = train_data_input.values.reshape(-1,IMG_SIZE,IMG_SIZE,1)
train_data_output = train_data.iloc[:,0]

X_test = test_data.values.reshape(-1,IMG_SIZE,IMG_SIZE,1)

In [6]:
train_data_input=train_data_input/255.0
X_test=X_test/255.0

In [7]:
X_train, X_val, y_train, y_val = train_test_split(train_data_input, train_data_output, test_size = 0.1, random_state = 42)

In [8]:
earlystopping = tf.keras.callbacks.EarlyStopping(monitor ="val_accuracy",
                              mode = 'auto', patience = 10,
                              restore_best_weights = True)

In [9]:
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)


In [10]:
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

In [11]:
with tpu_strategy.scope():
    model = tf.keras.Sequential() 
    
    model.add(tf.keras.layers.Conv2D(128,(5,5), input_shape=X_train.shape[1:]))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Activation('relu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2),strides=(2,2)))
    
    model.add(tf.keras.layers.Conv2D(256,(3,3)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Activation('relu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2),padding='same'))
    
    model.add(tf.keras.layers.Conv2D(512,(3,3)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Activation('relu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2),padding='same'))
    
    model.add(tf.keras.layers.Flatten())

    model.add(tf.keras.layers.Dense(540))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Activation('relu'))
    model.add(tf.keras.layers.Dropout(0.5))
    
    model.add(tf.keras.layers.Dense(80))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Activation('relu'))
    model.add(tf.keras.layers.Dropout(0.5))
    
    model.add(tf.keras.layers.Dense(10,activation='softmax'))
    
    model.compile(optimizer = 'adam',
      loss = 'sparse_categorical_crossentropy',
      metrics = ['accuracy'])

In [12]:
NUM_EPOCHS=50
BATCH_SIZE=32

history=model.fit(X_train,y_train , epochs=NUM_EPOCHS, batch_size=BATCH_SIZE, validation_data=(X_val,y_val),callbacks=[earlystopping],verbose=1)

Epoch 1/50
1182/1182 [==============================] - 19s 16ms/step - accuracy: 0.7945 - loss: 0.6951 - val_accuracy: 0.9681 - val_loss: 0.0987
Epoch 2/50
1182/1182 [==============================] - 16s 14ms/step - accuracy: 0.8893 - loss: 0.3932 - val_accuracy: 0.9805 - val_loss: 0.0614
Epoch 3/50
1182/1182 [==============================] - 16s 14ms/step - accuracy: 0.9107 - loss: 0.3268 - val_accuracy: 0.9771 - val_loss: 0.0710
Epoch 4/50
1182/1182 [==============================] - 16s 14ms/step - accuracy: 0.9246 - loss: 0.2771 - val_accuracy: 0.9860 - val_loss: 0.0431
Epoch 5/50
1182/1182 [==============================] - 16s 14ms/step - accuracy: 0.9303 - loss: 0.2563 - val_accuracy: 0.9821 - val_loss: 0.0521
Epoch 6/50
1182/1182 [==============================] - 17s 14ms/step - accuracy: 0.9372 - loss: 0.2356 - val_accuracy: 0.9905 - val_loss: 0.0322
Epoch 7/50
1182/1182 [==============================] - 16s 13ms/step - accuracy: 0.9378 - loss: 0.2273 - val_accuracy: 0.98

In [13]:
predictions = model.predict([X_test])
result = []
for i in range(len(predictions)):
    result.append(np.argmax(predictions[i]))

In [14]:
output = pd.DataFrame({ 'ImageId':[i+1 for i in test_data.index],
                        'Label':result
                    })
output.to_csv('./Kaggle_soln.csv', index=False)